In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../../py_files/')
import quadrop as qd

qd.set_plotting_style()

### Data pre-procesing

In [ ]:
# Example usage
base_dir = '../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/'
qd.consolidate_images(base_dir)

In [2]:
# Example usage
data_path = "../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/"

calibration_curve_paths = sorted(glob.glob("../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/calibration_curve/***ugml.tif"))


conditions_dict = {
    "A-RT": "Pos0", 
    "B-RT": "Pos1",
    "C-RT": "Pos2",
    "D-RT": "Pos3",
    "E-RT": "Pos4", 
    "F-RT": "Pos5",
    "G-RT": "Pos6",
    "H-RT": "Pos7",
    "negative": "Pos8"
}

# Organize PosX folders into condition folders
qd.organize_conditions(data_path, conditions_dict)

# Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)
time_interval_list = [40] * len(conditions)  # time intervals in seconds between frames for each condition
subconditions = ["Rep1"]
print("Conditions:", conditions)
print("Subconditions:", subconditions)

Conditions: ['A-RT', 'B-RT', 'C-RT', 'D-RT', 'E-RT', 'F-RT', 'G-RT', 'H-RT', 'negative']
Subconditions: ['Rep1']


In [ ]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


### Generate movies

In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    time_interval_list=time_interval_list, 
    vmax=20, 
    skip_frames=1, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    frame_rate=120,
    skip_frames=1
    )


In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions[:-1], 
    subconditions, 
    channel='cy5', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=120,
    batch_size=50
    )


In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    time_interval_list=time_interval_list, 
    vmax=500, 
    skip_frames=1, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=True,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    frame_rate=120,
    skip_frames=1
    )


In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions[:-1], 
    subconditions, 
    channel='GFP', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=120,
    batch_size=50
    )


In [ ]:
# qd.delete_temporary_image_directories(data_path, conditions, subconditions)

### Fluorescence Quantification

In [4]:
# Example usage
mw_kda_list = [44.5, 44.4, 44.9, 44.4, 64.9, 64.4, 65, 64.4, 50] 
droplet_volume_list = [2] * len(conditions)
protein_lengths_list = [401, 397, 401, 397, 596, 592, 596, 592, 500] 

qd.quantify_tiffiles(
    data_path, 
    conditions, 
    subconditions, 
    calibration_curve_paths, 
    mw_kda_list, 
    droplet_volume_list, 
    time_interval_list, 
    protein_lengths_list,
    ribosome_count=10**9,
    skip_frames=1,
    subtract_negative=True
)

Calculating sample intensities: 100%|██████████| 4224/4224 [00:47<00:00, 89.73it/s] 


('../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/combined_expression.csv',
 '../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/mean_expression.csv')

### PIV pre-processing

In [ ]:
qd.split_tiffs(data_path, conditions[:-1], subconditions, channel='cy5', file_interval=1)

### PIV

In [5]:
# Define feature limits and other parameters
v = 2E-7
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)
skip_frames = 1 ### CHANGE THIS TO SKIP FRAMES


velocity_limits = (None, None)
other_limits = (None, None)


feature_limits = {
    # 'u [m/s]': (-v, v), 
    # 'v [m/s]': (-v, v), 
    # 'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    # 'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}


# Features for PCA and plotting
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [um/s]",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [um]", 
    "power [W]_mean",
    "work [J]",
]


In [6]:
time_interval_list = [40] * len(conditions)  # time intervals in seconds between frames for each condition


# Process PIV data
qd.process_piv_data(
    data_path, 
    ['A-RT', 'C-RT', 'D-RT', 'E-RT', 'H-RT'],
    subconditions, 
    feature_limits, 
    time_interval_list, 
    min_frame=0, 
    max_frame=None, 
    skip_frames=1, 
    plot_autocorrelation=False, 
    frame_rate=1, 
    heatmaps=False
    )


Processing PIV data:   0%|          | 0/5 [00:00<?, ?it/s]






























































































































































































































Processing PIV data:  20%|██        | 1/5 [00:35<02:23, 35.85s/it]




































































































































































































































































































































































































































































Processing PIV data:  40%|████      | 2/5 [01:47<02:51, 57.19s/it]

















































































































Processing PIV data:

In [ ]:
# Plot features and PCA
qd.plot_PIV_all(
    data_path, 
    ['A-RT', 'C-RT', 'D-RT', 'E-RT', 'G-RT', 'H-RT'],
    subconditions, 
    features_pca, 
    min_frame=0, 
    max_frame=None
    )


### Expression + PIV

In [7]:
# Combine the data and save it to the "output_data" directory
qd.combine_averaged_dataframes(data_path, conditions, subconditions)


Conditions:   0%|          | 0/9 [00:00<?, ?it/s]

'Averaged' folder not found for B-RT/Rep1. Skipping...


Conditions:  56%|█████▌    | 5/9 [00:00<00:00, 46.40it/s]

'Averaged' folder not found for F-RT/Rep1. Skipping...


'Averaged' folder not found for G-RT/Rep1. Skipping...


Conditions: 100%|██████████| 9/9 [00:00<00:00, 60.98it/s]


'Averaged' folder not found for negative/Rep1. Skipping...
Combined DataFrame saved to: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/combined_PIV.csv


In [8]:
qd.merge_expression_piv_data(data_path)


Merged DataFrame saved to: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/merged_expression_PIV.csv


In [11]:
# # Specify the columns you want to plot
x_column = "time (h)"  # Example x-axis column
y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

 
# Apply Gaussian smoothing with sigma_x=2 and sigma_y=1
qd.plot_columns(data_path, x_column, y_column, sigma_x=1, sigma_y=5, x_log=False, y_log=True, min_frame=0, max_frame=None)


Plot saved for condition A-RT at ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/expression_piv_plots/time_(h)_vs_velocity_magnitude_m_s_mean_A-RT.png
Plot saved for condition B-RT at ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/expression_piv_plots/time_(h)_vs_velocity_magnitude_m_s_mean_B-RT.png
Plot saved for condition C-RT at ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/expression_piv_plots/time_(h)_vs_velocity_magnitude_m_s_mean_C-RT.png
Plot saved for condition D-RT at ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/output_data/expression_piv_plots/time_(h)_vs_velocity_magnitude_m_s_mean_D-RT.

In [ ]:

# List of features for PCA
features_pca = [
    "vorticity [1/s]_mean",
    # "velocity magnitude [m/s]_mean",
    # "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    # "correlation length [m]_mean", 
    "power [W]_mean",
    # "work [J]_mean",
    'vector direction [degrees]_mean',
    "Protein Concentration_nM", 
]

# Run PCA and save plot (with all conditions and subconditions in the same plot)
qd.plot_pca_expression_piv(data_path, conditions=conditions, subconditions=subconditions, features=features_pca, sigma=1)


In [3]:

# Example usage
qd.delete_outputs(data_path, conditions, subconditions, output_dirs=None)


Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/A-RT/Rep1/dataframes_PIV
Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/A-RT/averaged
Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/C-RT/Rep1/dataframes_PIV
Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/C-RT/averaged
Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras100nM-RT/2p5ul_TMBmix-1ulDNA100nM_/D-RT/Rep1/dataframes_PIV
Deleted directory: ../../../../../../mnt/c/Users/Admin/Thomson Lab Dropbox/David Larios/activedrops/main/090124-chimeras10